In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
car = pd.read_csv('true_car_listings.csv')


In [3]:
car

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic
...,...,...,...,...,...,...,...,...
852117,63215,2017,9,Culver City,CA,YV1A22MK9H1013237,Volvo,S90T6
852118,72260,2017,3201,Englewood,NJ,YV4A22PL3H1186162,Volvo,XC90T6
852119,55999,2016,28941,Fort Collins,CO,YV4A22PL4G1000868,Volvo,XC90AWD
852120,60240,2017,3005,San Leandro,CA,YV4A22NLXH1006162,Volvo,V90


In [4]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852122 entries, 0 to 852121
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Price    852122 non-null  int64 
 1   Year     852122 non-null  int64 
 2   Mileage  852122 non-null  int64 
 3   City     852122 non-null  object
 4   State    852122 non-null  object
 5   Vin      852122 non-null  object
 6   Make     852122 non-null  object
 7   Model    852122 non-null  object
dtypes: int64(3), object(5)
memory usage: 52.0+ MB


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [6]:
x = car.iloc[:, [1,2,3,4,6,7]]

In [7]:
x

,Year,Mileage,City,State,Make,Model
0,2014,35725,El Paso,TX,Acura,ILX6-Speed
1,2013,19606,Long Island City,NY,Acura,ILX5-Speed
2,2013,48851,El Paso,TX,Acura,ILX6-Speed
3,2014,39922,Windsor,CO,Acura,ILX5-Speed
4,2016,22142,Lindon,UT,Acura,ILXAutomatic
...,...,...,...,...,...,...
852117,2017,9,Culver City,CA,Volvo,S90T6
852118,2017,3201,Englewood,NJ,Volvo,XC90T6
852119,2016,28941,Fort Collins,CO,Volvo,XC90AWD
852120,2017,3005,San Leandro,CA,Volvo,V90


In [8]:
y = car.iloc[:, 0]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=3)

In [10]:
ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(x[['City','State','Make','Model']])

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [11]:
transform = make_column_transformer((OneHotEncoder(handle_unknown='ignore',categories=ohe.categories_),['City','State','Make','Model']),
                                    (StandardScaler(),['Year','Mileage'])
                                    ,remainder='drop')

In [12]:
lr = LinearRegression()

In [13]:
pipe = make_pipeline(transform, lr)

In [14]:
pipe.fit(x_train, y_train)

,steps,"[('columntransformer', ...), ('linearregression', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehotencoder', ...), ('standardscaler', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [15]:
y_pred = pipe.predict(x_test)

In [16]:
y_pred

array([19723.65186252, 34860.87047634, 35410.58361211, ...,
       21900.22745639, 36713.93969376, 24826.1679475 ])

In [17]:
r2_score(y_test, y_pred)

0.8715228949740981

In [18]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = pipe.predict(x_test)

print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

R2 Score: 0.8715228949740981
MAE: 2851.816210602803


In [19]:
len(lr.coef_)

5408

In [20]:
y_pred = pipe.predict(x_test)
print("R2 Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

R2 Score: 0.8715228949740981
MAE: 2851.816210602803


In [21]:
# from sklearn.model_selection import cross_val_score
# cross_val_score(pipe, x, y, scoring='r2', cv=5).mean()

In [22]:
# import matplotlib.pyplot as plt
# import seaborn as sns

In [23]:
# plt.figure(figsize=(12, 8))
# sns.scatterplot(x=y_test, y=y_pred)
# plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
# color = 'red', linestyle='--')
# plt.xlabel('Actual Price')
# plt.ylabel('Predicted Price')
# plt.show()

In [24]:
# sns.histplot(data=car, x = 'Price', bins = 30,kde = True)
# plt.show()

In [25]:
# sns.scatterplot(x=y_test, y=y_pred)

In [26]:
# for i in range(15):
#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=i)
#     lr = LinearRegression()
#     pipe = make_pipeline(transform, lr)
#     pipe.fit(x_train, y_train)
#     y_pred = pipe.predict(x_test)
#     print("R2 Score:",i,'-', r2_score(y_test, y_pred))

In [27]:
from sklearn.metrics import r2_score

# Regular R² score
r2 = r2_score(y_test, y_pred)

# n = number of samples, p = number of features
n = x_test.shape[0]
p = x_test.shape[1]

# Adjusted R² formula
adjusted_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

print("R² Score:", r2)
print("Adjusted R² Score:", adjusted_r2)


R² Score: 0.8715228949740981
Adjusted R² Score: 0.871518371610192


In [28]:
x_test.shape[0]

170425

In [29]:
x_test.shape[1]

6

In [30]:
x_test

,Year,Mileage,City,State,Make,Model
461287,2015,43947,Seekonk,MA,Hyundai,Santa
7586,2018,10,Langhorne,PA,Acura,RDXAWD
816201,2015,20445,Manchester,NH,Toyota,Tundra
492460,2016,9889,Latham,NY,Jeep,RenegadeJustice
575571,2014,47479,Marietta,GA,Lincoln,MKZFWD
...,...,...,...,...,...,...
815517,2016,17230,Edmonds,WA,Toyota,HighlanderAWD
153725,2014,46373,Durant,OK,Chevrolet,Silverado
121466,2010,95591,Muscle Shoals,AL,Chevrolet,Silverado
316969,2016,19545,Price,UT,Ford,F-1504WD


In [31]:
pipe.predict(pd.DataFrame([[2010,50000,'Seekonk','GA','Ford','EscapeSE']], columns=['Year','Mileage','City','State','Make','Model']))

array([10049.79506366])

In [34]:
import pickle
pickle.dump(pipe, open('LinearRegrationModel.pkl', 'wb'))